In [1]:
# dependencies
import pymongo as pm
import urllib
from scrape import Scrape as sc
from pprint import pprint
from bs4 import BeautifulSoup as bs
import re
import time

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pm.MongoClient(conn)

In [20]:
# define db and collections
DB_NAME = 'scrape'
db = client[DB_NAME]

# this collection will store all the HTML pages
COL_NAME = 'col_html'
htmlCollection = db[COL_NAME]

In [21]:
# import db into local mongo instance

PATH = 'drive/NR_DailyB_Intercept_VG.json'

!mongoimport --db {DB_NAME} --collection {COL_NAME} --file {PATH} --batchSize 1

2018-06-16T11:03:08.047-0400	connected to: localhost
2018-06-16T11:03:11.034-0400	[##########..............] scrape.col_html	369MB/828MB (44.5%)
2018-06-16T11:03:14.034-0400	[####################....] scrape.col_html	710MB/828MB (85.7%)
2018-06-16T11:03:15.205-0400	[########################] scrape.col_html	828MB/828MB (100.0%)
2018-06-16T11:03:15.205-0400	imported 4768 documents


In [6]:
# startURL = 'http://books.toscrape.com/'
# domain = 'toscrape.com'

# declare top-level url and domain
# startURL = 'https://billypenn.com/'
# domain = 'billypenn.com'

# startURL = 'https://theintercept.com/'
# domain = 'theintercept.com'

startURL = 'http://www.motherjones.com/'
domain = 'www.motherjones.com'

# establish necessary data structures
done_urls = set()
external_urls = []
urls = set()
pagelinks = set()

# seed with top-level url of website
urls.add(startURL)

In [7]:
# counter for limiting functions
ctr = 0

# continue to loop through urls until we have scraped all valid
# needed a while loop so we can modify size of urls through process
while (len(urls) > 0) and ctr < 2:    #(len(done_urls) < 200):
    
    # timer for performance across url loops
    forStart = time.time()
    
    # go through every url that has not been scraped yet
    for url in urls:
        print("URL in for loop: ", url)
        
        # timer for performance
        start = time.time()
        
        # governor to avoid being banned/blocked
        time.sleep(5)   # time governor
        
        # check for email and run all other links through scrape
        html = sc.scrapeHTML(url)
        
        # end timer for performance and print status
        end = time.time()
        print(f'Elapsed time: {end - start}')
        
        # validate html and then add to the pagelinks all the links that were scraped
        if html: 
            #print(f'Html scraped: {html[:50]}')
            data = {
                "url": url, 
                "html": html, 
                }

            # insert dict into db document
            htmlCollection.insert_one(data)
            
            # record url in "done" collection
            done_urls.add(url)
            
            pagelinks = pagelinks.union(sc.scrapeURL(html, url))
            print(f'Extracted links: {len(pagelinks)}')
        else:
            print(f'Main Loop: HTML not extracted')
        
        ####################################################
        # TODO: update done_urls, external and urls to CSV #
        ####################################################
        
    # end timer of for loop
    forEnd = time.time()
    
    # print status for testing
    print()
    print(f'Number of Urls: {len(urls)}')
    print(f'Process Time: {forEnd -forStart}')
    print('--------------------------------------------------------------')
        
        
    
    # add links from page to urls set outside of for loop
    if html:
        for link in pagelinks:
            if domain in link:
                urls.add(link)
                # print(f"Link in pagelinks: {link}")
            else:
                external_urls.append(link)
    
    # remove done_urls from todo urls and increment counter
    urls -= done_urls
    ctr += 1
    
    # print status for testing
    print("done_urls: ", len(done_urls))
    print("(urls - done_urls): ", len(urls))
    print("LOOP counter: ", ctr)
    print("--------------------")


URL in for loop:  http://www.motherjones.com/
ScrapeHTML: HTTPConnectionPool(host='www.motherjones.com', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000020D61E845C0>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',))
Elapsed time: 26.012712717056274
Main Loop: HTML not extracted

Number of Urls: 1
Process Time: 26.012712717056274
--------------------------------------------------------------
done_urls:  0
(urls - done_urls):  1
LOOP counter:  1
--------------------
URL in for loop:  http://www.motherjones.com/
Elapsed time: 8.270049333572388
Extracted links: 116

Number of Urls: 1
Process Time: 8.381651878356934
--------------------------------------------------------------
done_urls:  1
(urls - done_urls):  97
LOOP coun